# 3

In [99]:
c= []
for i in range(5):
    print('day', i+1)
    x = np.random.choice([6, 8, 10, 12], p=[0.25, 0.35, 0.3, 0.1])
    print('# of customers:', x)
    d = []
    for j in range(x):
        y = np.random.choice([1, 2, 3, 4], p=[0.35, 0.45, 0.15, 0.05])
        d += [y]
    print('# of breads:', d,'\n')
    c += [sum(d)]
    
sum(c)/5

day 1
# of customers: 10
# of breads: [1, 2, 2, 2, 2, 1, 3, 1, 1, 2] 

day 2
# of customers: 8
# of breads: [2, 2, 3, 1, 2, 1, 3, 2] 

day 3
# of customers: 8
# of breads: [1, 1, 3, 2, 2, 3, 1, 2] 

day 4
# of customers: 8
# of breads: [3, 4, 2, 2, 2, 2, 2, 3] 

day 5
# of customers: 6
# of breads: [1, 2, 4, 2, 1, 3] 



16.2

# 4

A) Average of two random points distance on circle

In [7]:
import random
from math import sqrt

In [10]:
N = 1000000
distances = list()

def point_generator():
    x = random.random()
    y = sqrt(1 - (x * x))
    negative = random.randint(0, 1)
    if negative:
        y = -y
    return x, y

for i in range(N):
    x_1, y_1 = point_generator()
    x_2, y_2 = point_generator()
    
    distance = sqrt((x_2 - x_1) * (x_2 - x_1) + (y_2 - y_1) * (y_2 - y_1))
    distances.append(distance)

mean = sum(distances) / N
mean

1.0236094128051214

B) General Answer

$$
average \: value \: d(\theta) = \frac{1}{\pi - 0}\int_{0}^{\pi}2sin\frac{\theta}{2}d\theta = \frac{1}{\pi}(-4cos(\frac{\theta}{2}))_0^{\pi} = \frac{4}{\pi} = 1.27
$$

# 5

A) Python

C) We can generate a random number using `np.random.choice` method. It can be seen in the code.

In [1]:
NUMBER_OF_TRUCKS, NUMBER_OF_LOADERS = 9, 2
t, MAX_TRAVELS_NUMBER = 0, 100

## States

In [2]:
LOADING_QUEUE = 1
LOADING = 2
WEIGHING_QUEUE = 3
WEIGHING = 4
TRAVELING = 5

In [4]:
traveling_probabilities = [0.4, 0.3, 0.2, 0.1]
traveling_times = [40, 60, 80, 100]

weighing_probabilities = [0.7, 0.3]
weighing_times = [12, 16]

loading_probabilities = [0.3, 0.5, 0.2]
loading_times = [5, 10, 15]

## Truck Class

In [123]:
class Truck:
    def __init__(self, plate):
        self.plate = plate
        self.state = LOADING_QUEUE
        self.traveling_time = 0
        self.spent_time = 0
    
    
    def traveling(self):
        random = np.random.choice(traveling_times, p=traveling_probabilities, size=(1))[0]
        self.spent_time = 0
        self.traveling_time = random
        self.state = TRAVELING
    
    
    def ended(self):
        if self.spent_time == self.traveling_time:
            print("truck {} ended traveling at {}".format(self.plate, t))
            self.traveling_time = 0
            self.spent_time = 0
            return True
        
        return False

## Scaler class

In [131]:
class Scaler:
    def __init__(self):
        self.truck = None
        self.weighing_time = 0
        self.spent_time = 0
    
    
    def add_truck(self, truck):
        if truck:
            self.weighing_time = np.random.choice(weighing_times, p=weighing_probabilities, size=(1))[0]
            self.truck = truck
            self.spent_time = 0
            truck.state = WEIGHING
            print("truck {} is weighing for {}".format(truck.plate, self.weighing_time))
        
    def ended(self):
        if self.spent_time == self.weighing_time:
            print("truck {} ended weighing at {}".format(self.truck.plate, t))
            truck = self.truck
            self.weighing_time = 0
            self.spent_time = 0
            self.truck = None
            return truck
        
        return None

## Loader class

In [130]:
class Loader:
    def __init__(self, id):
        self.truck = None
        self.id = id
        self.loading_time = 0
        self.spent_time = 0
        
        
    def add_truck(self, truck):
        if truck:
            self.loading_time = np.random.choice(loading_times, p=loading_probabilities, size=(1))[0]
            self.spent_time = 0
            self.truck = truck
            truck.state = LOADING
            print("loading truck {} with loader at {}".format(truck.plate, self.id, self.loading_time))
    
    
    def ended(self):
        if self.spent_time == self.loading_time:
            print("loader {}: loading of truck {} finished at {}".format(self.id, truck.plate, t))
            truck = self.truck
            self.truck = None
            self.spent_time = 0
            self.loading_time = 0
            return truck
        
        return None

## Initializing

In [ ]:
def get_all_trucks():
    return [Truck(i) for i in range(NUMBER_OF_TRUCKS)]


def get_all_loaders():
    return [Loader(i) for i in range(NUMBER_OF_LOADERS)]

trucks = get_all_trucks()
loaders = get_all_loaders()
scaler = Scaler()

number_of_travels = 0
loading_queue, weighting_queue = list(), list()

for truck in trucks:
    loading_queue.append(truck)
    

In [134]:
while number_of_travels <= MAX_TRAVELS_NUMBER:
    for truck in trucks:
        if truck.state == TRAVELING:
            truck.spent_time += 1
            if truck.ended():
                number_of_travels += 1
                loading_queue.append(truck)
                truck.state = LOADING_QUEUE
    
    for loader in loaders:
        if loader.truck:
            loader.spent_time += 1
            truck_ended = loader.ended()
            if truck_ended:
                weighting_queue.append(truck_ended)
                truck_ended.state = WEIGHING_QUEUE
            if loader.truck == None:
                if len(loading_queue):
                    loader.add_truck(loading_queue.pop(0))
                    
    if scaler.truck:
        scaler.spent_time += 1
        truck_ended = scaler.ended()
        if truck_ended:
            truck.traveling()
    else:
        if len(weighting_queue):
            scaler.add_truck(weighting_queue.pop(0))
    t += 1

0